In [2]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd

# import statsmodels sm,ols and pairwise_tukeyhsd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [8]:
# save the state of randomness
np.random.seed(123)

# generate an array of eight made-up cookie weights for each bakery from random 
# Gaussian distribution with specified means and standard 
# deviations (note, assume means and standard deviation are for samples)
bakery_a = np.random.normal(26,3,8).round(1)
bakery_b = np.random.normal(25,3,8).round(1)
bakery_c = np.random.normal(28,1,8).round(1)

# print out the individual cookie weights for each bakery
print(f'bakery_a: {bakery_a}\nbakery_b: {bakery_b}\nbakery_c: {bakery_c}')

bakery_a: [22.7 29.  26.8 21.5 24.3 31.  18.7 24.7]
bakery_b: [28.8 22.4 23.  24.7 29.5 23.1 23.7 23.7]
bakery_c: [30.2 30.2 29.  28.4 28.7 29.5 27.1 29.2]


In [18]:
# incorporate bakery_id and cookie weights into pandas DataFrame
# where cookies arrays are sequentially concatenated into a single array
bakeries_df = pd.DataFrame({'bakery_id' : ['a']*8 + ['b']*8 + ['c']*8,
                             'weights_g' : np.concatenate((bakery_a,
                                                          bakery_b,
                                                          bakery_c),axis=0)})


In [20]:
# create a model with statsmodels ols and fit data into the model
model = ols(formula='weights_g ~ bakery_id', data=bakeries_df).fit()

# call statsmodels stats.anova_lm and pass model as an argument to perform ANOVA
anova_result = sm.stats.anova_lm(model, type=2)

print(anova_result)

             df      sum_sq    mean_sq         F    PR(>F)
bakery_id   2.0   93.523333  46.761667  5.694829  0.010569
Residual   21.0  172.436250   8.211250       NaN       NaN


In [28]:
# call statsmodels' pairwise_tukeyhsd to perform multiple comparisons test
# assign bakeries_df weights_g and bakery_id columns and set alpha to 0.05
multiple_comp_result = pairwise_tukeyhsd(endog=bakeries_df['weights_g'],
                                         groups=bakeries_df['bakery_id'],
                                        alpha=0.05)
print(multiple_comp_result.summary())

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     a      b    0.025    0.9 -3.5854 3.6354  False
     a      c      4.2 0.0208  0.5896 7.8104   True
     b      c    4.175 0.0217  0.5646 7.7854   True
---------------------------------------------------
